# CADS API: filters and pagination tests

In [1]:
import os
import xarray as xr

import cads_api_client

## Client instantiation

In [2]:
api_url = os.getenv("CADS_API_ROOT_URL", "http://cds2-dev.copernicus-climate.eu/api")
api_url

'http://cds2-dev.copernicus-climate.eu/api'

In [3]:
client = cads_api_client.ApiClient(url=api_url, key="00112233-4455-6677-c899-aabbccddeeff")

## 1. Catalogue Exploration

### 1.1 Fetch all available collections

**Objective**: verify the access to the complete list of **collections**
<hr>

_**Expected result**:  client.collections() returns all available collections in the CDS_

In [4]:
client.collections().collection_ids()

['cams-global-reanalysis-eac4-monthly',
 'reanalysis-era5-single-levels',
 'reanalysis-era5-land',
 'reanalysis-era5-pressure-levels',
 'reanalysis-era5-land-monthly-means',
 'derived-near-surface-meteorological-variables']

### 1.2 Pagination

**Objective**: verify that returned **`collections`** can be paginated in groups of specified lenght. The user can move between pages usign **`next()`** and **`prev()`** methods
<hr>

_**Expected result**:  **`collections`** are paginated in groups of lenght `[limit]`. Calling **`next()`** moves to the following page. Calling **`prev()`** moves to the previous page. When the limits are reached **`next()`** and **`prev()`** return **`None`**_

In [5]:
collections = client.collections(limit=1)
len(collections.json['collections'])

1

In [6]:
collections.json['collections'][0]['id']

'cams-global-reanalysis-eac4-monthly'

Use **`next()`** to move to the followgin page

In [7]:
collections.next().json['collections'][0]['id']

'reanalysis-era5-single-levels'

In [8]:
collections.next().next().next().next().next().json['collections'][0]['id']

'derived-near-surface-meteorological-variables'

use **`prev()`** to move to the previus page

In [9]:
collections.next().prev().json['collections'][0]['id']

'cams-global-reanalysis-eac4-monthly'

when the limit is reached **`next()`** return None

In [10]:
collections.next().next().next().next().next().next() is None

True

so does **`prev()`**

In [11]:
collections.prev() is None

True

Try a different `limit` for page 

In [12]:
collections = client.collections(limit=4)
collections.collection_ids()

['cams-global-reanalysis-eac4-monthly',
 'reanalysis-era5-single-levels',
 'reanalysis-era5-land',
 'reanalysis-era5-pressure-levels']

In [13]:
collections.next().collection_ids()

['reanalysis-era5-land-monthly-means',
 'derived-near-surface-meteorological-variables']

### 1.3 Filter by full text search in collections' tile

**Objective**: verify that the catalogue can be filtered by full text search in collections' title
<hr>

_**Expected result**:  only **`collections`** with the specified full text `q` in the title are returned._

In [14]:
collections = client.collections(q="hourly")
collections.collection_ids()

['reanalysis-era5-single-levels',
 'reanalysis-era5-land',
 'reanalysis-era5-pressure-levels']

In [15]:
for collection in collections.json['collections']:
    print(collection['title'])

ERA5 hourly data on single levels from 1959 to present
ERA5-Land hourly data from 1950 to present
ERA5 hourly data on pressure levels from 1959 to present


### 1.4 Filter by keywords
**Objective**: verify that the catalogue can be filtered by searching between datasets' keywords
<hr>

_**Expected result**:  only **`collections`** with the specified kewords `kw` are returned_

In [16]:
client.collections(kw=["Temporal coverage: Past", "Spatial coverage: Global"]).collection_ids()

['cams-global-reanalysis-eac4-monthly',
 'reanalysis-era5-single-levels',
 'reanalysis-era5-land',
 'reanalysis-era5-pressure-levels',
 'reanalysis-era5-land-monthly-means',
 'derived-near-surface-meteorological-variables']

In [17]:
client.collections(kw=["Product type: Reanalysis"]).collection_ids()

['cams-global-reanalysis-eac4-monthly',
 'reanalysis-era5-single-levels',
 'reanalysis-era5-land',
 'reanalysis-era5-pressure-levels',
 'reanalysis-era5-land-monthly-means']

### 1.4 Sort collections by id, title or update
**Objective**: verify that **collections** can be sorted by `id`, `title` or `update`
<hr>

_**Expected result**:  returned **`collections`** are sorted by `id`, `title` or `update`

In [18]:
client.collections(sorting="id").collection_ids()

['cams-global-reanalysis-eac4-monthly',
 'derived-near-surface-meteorological-variables',
 'reanalysis-era5-land',
 'reanalysis-era5-land-monthly-means',
 'reanalysis-era5-pressure-levels',
 'reanalysis-era5-single-levels']

In [19]:
client.collections(sorting="title").collection_ids()

['cams-global-reanalysis-eac4-monthly',
 'reanalysis-era5-pressure-levels',
 'reanalysis-era5-single-levels',
 'reanalysis-era5-land',
 'reanalysis-era5-land-monthly-means',
 'derived-near-surface-meteorological-variables']

In [20]:
titles = [collection["title"] for collection in client.collections().json['collections']]
sorted(titles)

['CAMS global reanalysis (EAC4) monthly averaged fields',
 'ERA5 hourly data on pressure levels from 1959 to present',
 'ERA5 hourly data on single levels from 1959 to present',
 'ERA5-Land hourly data from 1950 to present',
 'ERA5-Land monthly averaged data from 1950 to present',
 'Near surface meteorological variables from 1979 to 2019 derived from bias-corrected reanalysis']

In [21]:
client.collections(sorting="update").collection_ids()

['cams-global-reanalysis-eac4-monthly',
 'reanalysis-era5-single-levels',
 'reanalysis-era5-land',
 'reanalysis-era5-pressure-levels',
 'reanalysis-era5-land-monthly-means',
 'derived-near-surface-meteorological-variables']

### 1.5 A combination of the above
**Objective**: verify that **collections** can be filtered by keyword, full text search in collections' title, then sorted by `title` and finally paginated.
<hr>

_**Expected result**:  returned **`collections`** are filtered by keyword, title text, sorted by `id` and paginated_

In [22]:
collections = client.collections(
    limit=3, 
    kw='Product type: Reanalysis', 
    q='era5', 
    sorting='id'
)
collections.collection_ids()

['reanalysis-era5-land',
 'reanalysis-era5-land-monthly-means',
 'reanalysis-era5-pressure-levels']

In [23]:
collections.next().collection_ids()

['reanalysis-era5-single-levels']